In [1]:
import pandas as pd
import gzip
import json
import pyperclip
import numpy as np
import os

In [2]:
from openai import OpenAI
client = OpenAI()

In [3]:
df = pd.read_csv('yelp-100kv2.csv')
#df['rating'] = df['rating'].astype(int)
#df['price'] = df['price'].apply(lambda x: None if not str(x).startswith('$') else x)
#df.reset_index(drop=True, inplace=True)
#df.columns = ['reviewText']
df.head()

,business_id,stars,text,name,categories
0,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,7ATYjTIgM3jUlt4UM3IypQ,5.0,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,YjUWPpI6HXG530lwP-fb2A,3.0,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,"Restaurants, Breakfast & Brunch"
3,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",Zaika,"Halal, Pakistani, Restaurants, Indian"
4,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...,Melt,"Sandwiches, Beer, Wine & Spirits, Bars, Food, ..."


In [4]:
df['text'] = df['text'].replace('\n', '', regex=True)
df.rename(columns={'text': 'reviewText'}, inplace=True)
df.drop('business_id', axis=1, inplace=True)
df['stars']=df['stars'].astype(int) 
df.head()


,stars,reviewText,name,categories
0,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,5,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,3,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,"Restaurants, Breakfast & Brunch"
3,5,"Wow! Yummy, different, delicious. Our favo...",Zaika,"Halal, Pakistani, Restaurants, Indian"
4,4,Cute interior and owner (?) gave us tour of up...,Melt,"Sandwiches, Beer, Wine & Spirits, Bars, Food, ..."


In [67]:
df['reviewText_length'] = df['reviewText'].astype(str).apply(len)
top_10_longest = df.nlargest(10, 'reviewText_length')
print(top_10_longest[['reviewText', 'reviewText_length']])
df.drop('reviewText_length', axis=1, inplace=True)

                                              reviewText  reviewText_length
66144  I first heard of Spice Kitchen after visiting ...               4999
49421  I remembered going to this museum as a kid, bu...               4991
48570  OK, I've only been a Yelper since March, and f...               4980
89356  WATCH YOUR BILL!! The laminated menus have pri...               4979
6410   Ohmygod ohmygod ohmygod ohmygod!  I have a new...               4976
57100  Sticky floors and flat drinks.... My wife and ...               4972
55529  First off, let me just say how good the food w...               4971
79211  I took my mother and daughter here on Easter S...               4970
32863  Deplorable. This is how I would describe the c...               4968
73460  I'd been to the Prime Rib just once before com...               4967


In [11]:
tools = [
    {
        'name': 'write_review',
        'description': 'Write a review',
        'parameters': {
            'type': 'object',
            'properties': {
                'Review': {
                    'type': 'string',
                    'Review': 'Review description.'
                }
            }
        }
    }
]

In [74]:
nr = 1
min_length = 0 if len(df['reviewText'][nr])-20 < 0 else len(df['reviewText'][nr])-20
max_length = len(df['reviewText'][nr])+20
#Craft a product review that convincally mimics an authentic consumer's perspective. Your aim is to create a review that blends seamlessly with legitimate reviews. Refer to the provided product details but select only certain elements that seem most relevant or noteworthy to include in your narrative, akin to how a real customer might mention only aspects that stood out to them. Provided please find an example real review text you can use as reference. Ensure your review reflects a rating of {df['rating'][nr]} stars and the review length is no longer than {len(df['reviewText'][nr])} characters.
prompt6 = f"""
Write a review for visited business. The review should be crafted to appear authentic, using a human-like language that mimics a genuine customer testimonial.
Use to the provided business details for context.  Consider the example review as reference how a similar review looks like.
Ensure your review reflects a rating of {df['stars'][nr]} stars and the review length is between {min_length} and {max_length} characters.

Example Review for Reference:
"{df['reviewText'][nr]}"

Business Information for context:
- Categories: '{df['categories'][nr]}'
"""

messages = []
messages.append({"role": "system", "content": "Your task is to create a review that could easily be mistaken for a genuine user-written review. The review should integrate elements of style, tone, and linguistic nuances found in real consumer testimonials to ensure it blends seamlessly with legitimate content."})
messages.append({"role": "user", "content": prompt6})

#promptujemy
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=messages,
  functions = tools,
  function_call = {"name": "generate_fake_review"}
)

print(prompt6)
response_mess = json.loads(response.choices[0].message.function_call.arguments)
review_text = response_mess['Review']
print(review_text)
#wygenerowane.append(review_text)


Write a review for visited business. The review should be crafted to appear authentic, using a human-like language that mimics a genuine customer testimonial.
Use to the provided business details for context.  Consider the example review as reference how a similar review looks like.
Ensure your review reflects a rating of 5 stars and the review length is between 805 and 845 characters.

Example Review for Reference:
"I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycle. From the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.For anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gyms make you do).There is no way I can write this review without giving Russell, the owner of Body Cycle, a shout out. Russell's passion for fitness and cycling is so evident, as is his desi

In [6]:
# Dodaj nową kolumnę do df, która będzie przechowywać wygenerowane recenzje
df['Wygenerowane'] = ''

In [15]:
import time

start_time = time.time()

range_of_numbers = range(80500, 81001)  # Replace with your desired range of numbers
for nr in range_of_numbers:
  error_count = 0
  while True:
      try:
        if isinstance(df['reviewText'][nr], str):
          min_length = 0 if len(df['reviewText'][nr])-20 < 0 else len(df['reviewText'][nr])-20
          max_length = len(df['reviewText'][nr])+20
        else:
         min_length = 0
         max_length = 20

        prompt6 = f"""
        Write a review for a visited restaurant. Ensure your review reflects a rating of {df['stars'][nr]} stars and the review length is between {min_length} and {max_length} characters.
        """

        messages = []
        #messages.append({"role": "system", "content": "Your task is to create a review that could easily be mistaken for a genuine user-written review. The review should integrate elements of style, tone, and linguistic nuances found in real consumer testimonials to ensure it blends seamlessly with legitimate content."})
        messages.append({"role": "user", "content": prompt6})

        response = client.chat.completions.create(
          model="gpt-3.5-turbo-0125",
          messages=messages,
          functions = tools,
          function_call = {"name": "write_review"}
        )

        response_mess = json.loads(response.choices[0].message.function_call.arguments)
        review_text = response_mess['Review']
            
        # Dodaj wygenerowaną recenzję do kolumny 'Wygenerowane' w df
        df.loc[nr, 'Wygenerowane'] = review_text

        # Jeśli nie ma błędu, przerwij pętlę while
        break
      except KeyError:
        error_count += 1
        #Na wypadek gdyby wychodziły wielokrotne błedy
        if error_count > 3:
          print(f'Error for {nr} record')
          break
        continue
  if nr % 100 == 0:
    print(f'Processed {nr} record')
    execution_time = time.time() - start_time
    minutes = execution_time // 60
    seconds = execution_time % 60
    print(f"Czas wykonania: {int(minutes)} minut {seconds} sekund")


Processed 80500 record
Czas wykonania: 0 minut 2.1948628425598145 sekund
Processed 80600 record
Czas wykonania: 2 minut 56.61778116226196 sekund
Processed 80700 record
Czas wykonania: 5 minut 47.92845702171326 sekund
Processed 80800 record
Czas wykonania: 8 minut 50.29943895339966 sekund
Processed 80900 record
Czas wykonania: 12 minut 8.49399995803833 sekund
Processed 81000 record
Czas wykonania: 15 minut 24.939981937408447 sekund


In [13]:
for i in range(80000,80020):
    print(str(i)+' OG:'+df.loc[i, 'reviewText'])
    print(str(i)+' CG:'+df.loc[i, 'Wygenerowane'])


80000 OG:They advertise themselves as"Authentic Creole Cuisine.  Cajun and Creole Spices"Every kind of hot sauce-baby bottles or medium or large....funny names too...like the Obama Hot Sauce which states, "Changing The Country One Drop at a Time!"Hurricane mix to bring home, seafood boil with a play on a raccoon and alligator throwing a crawfish in a pot (as they are also cooked in spices in a pot in Louisiana too!)Playing cards with drink recipes, crawfish pencils.....need I continue-everything is fun albeit a bit expensive.It is located in the French Market and has an eating area attached.
80000 CG:I had a delightful dining experience at this charming restaurant. The ambiance was cozy and inviting, perfect for a casual meal or a special occasion. The staff was attentive and friendly, making sure we were well taken care of throughout our visit. The menu offered a good variety of options, and the food was delicious. I particularly enjoyed the flavorful dishes and the presentation was t

In [18]:
reviews_generated4_full = df.reset_index()
reviews_generated4_full['Wygenerowane'] = reviews_generated4_full['Wygenerowane'].replace('', np.nan)
reviews_generated4_full['text_'] = np.where(reviews_generated4_full['Wygenerowane'].notnull(), reviews_generated4_full['Wygenerowane'], reviews_generated4_full['reviewText'])
reviews_generated4_full['label'] = np.where(reviews_generated4_full['Wygenerowane'].notnull(), 'CG', 'OG')
reviews_generated4_full['target'] = np.where(reviews_generated4_full['Wygenerowane'].notnull(), 1, 0)

#[['index','reviewText', 'Wygenerowane']]
print(reviews_generated4_full)
reviews_generated4_full.to_excel('reviews_generated4_full.xlsx', index=True)
reviews_generated4_full.to_csv('reviews_generated4_full.csv', index=True)

       index  stars                                         reviewText  \
0          0      3  If you decide to eat here, just be aware it is...   
1          1      5  I've taken a lot of spin classes over the year...   
2          2      3  Family diner. Had the buffet. Eclectic assortm...   
3          3      5  Wow!  Yummy, different,  delicious.   Our favo...   
4          4      4  Cute interior and owner (?) gave us tour of up...   
...      ...    ...                                                ...   
99995  99995      5  Came in for my 5-6 month prophy and saw Kara -...   
99996  99996      3  Why 3 stars?I love Target, but here is just so...   
99997  99997      4  The Local Taco is the first joint I've found i...   
99998  99998      5  Top notch! He will cater our wedding and  Fran...   
99999  99999      5  Best place to get a vietnamese sandwich and bo...   

                                   name  \
0          Turning Point of North Wales   
1            Body Cycle S

In [22]:
reviews_generated4_cut = reviews_generated4_full[['text_', 'label', 'target']].iloc[80000:82000]
reviews_generated4_cut.columns = ['text', 'label', 'target']
reviews_generated4_cut.to_csv('reviews_generated4_cut.csv', index=True)
reviews_generated4_cut.to_excel('reviews_generated4_cut.xlsx', index=True)
print(reviews_generated4_cut)

                                                    text label  target
80000  I had a delightful dining experience at this c...    CG       1
80001  The restaurant had a cozy ambiance and friendl...    CG       1
80002  I recently visited this restaurant and I must ...    CG       1
80003  The restaurant had good ambience and friendly ...    CG       1
80004  I recently visited this restaurant and had a w...    CG       1
...                                                  ...   ...     ...
81995  Super Gyros and More was one of my absolute fa...    OG       0
81996  Sal's is one of our favorites. We are always m...    OG       0
81997  December 13-Nick, Initial contact-easy. Initia...    OG       0
81998  The best waterfront view and best onion rings,...    OG       0
81999  Stopped in for lunch because of a "Groupon now...    OG       0

[2000 rows x 3 columns]


In [ ]:
if not os.path.exists('reviews_generated2_test.csv'):
    Wygenerowane.to_csv('reviews_generated2_test.csv', index=True)
print(reviews_generated1_cut)

In [80]:
df[:100].to_excel('yelp-test.xlsx', index=True)


### GPT4

In [27]:
import time

start_time = time.time()

range_of_numbers = range(70100, 70501)  # Replace with your desired range of numbers
for nr in range_of_numbers:
  error_count = 0
  while True:
      try:
        if isinstance(df['reviewText'][nr], str):
          min_length = 0 if len(df['reviewText'][nr])-20 < 0 else len(df['reviewText'][nr])-20
          max_length = len(df['reviewText'][nr])+20
        else:
         min_length = 0
         max_length = 20

        prompt6 = f"""
        Write a review for a visited restaurant. Ensure your review reflects a rating of {df['stars'][nr]} stars and the review length is between {min_length} and {max_length} characters.
        """

        messages = []
        #messages.append({"role": "system", "content": "Your task is to create a review that could easily be mistaken for a genuine user-written review. The review should integrate elements of style, tone, and linguistic nuances found in real consumer testimonials to ensure it blends seamlessly with legitimate content."})
        messages.append({"role": "user", "content": prompt6})

        response = client.chat.completions.create(
          model="gpt-4-turbo",
          messages=messages,
          functions = tools,
          function_call = {"name": "write_review"}
        )

        response_mess = json.loads(response.choices[0].message.function_call.arguments)
        review_text = response_mess['Review']
            
        # Dodaj wygenerowaną recenzję do kolumny 'Wygenerowane' w df
        df.loc[nr, 'Wygenerowane'] = review_text

        # Jeśli nie ma błędu, przerwij pętlę while
        break
      except KeyError:
        error_count += 1
        #Na wypadek gdyby wychodziły wielokrotne błedy
        if error_count > 3:
          print(f'Error for {nr} record')
          break
        continue
  if nr % 10 == 0:
    print(f'Processed {nr} record')
    execution_time = time.time() - start_time
    minutes = execution_time // 60
    seconds = execution_time % 60
    print(f"Czas wykonania: {int(minutes)} minut {seconds} sekund")


Processed 70100 record
Czas wykonania: 0 minut 7.3488078117370605 sekund
Processed 70110 record
Czas wykonania: 0 minut 57.26360106468201 sekund
Processed 70120 record
Czas wykonania: 1 minut 43.059962034225464 sekund
Processed 70130 record
Czas wykonania: 2 minut 42.47720289230347 sekund
Processed 70140 record
Czas wykonania: 3 minut 22.62781310081482 sekund
Processed 70150 record
Czas wykonania: 4 minut 12.587984085083008 sekund
Processed 70160 record
Czas wykonania: 5 minut 0.30651402473449707 sekund
Processed 70170 record
Czas wykonania: 6 minut 0.28081822395324707 sekund
Processed 70180 record
Czas wykonania: 6 minut 41.47335696220398 sekund
Processed 70190 record
Czas wykonania: 8 minut 3.4138638973236084 sekund
Processed 70200 record
Czas wykonania: 9 minut 15.184715986251831 sekund
Processed 70210 record
Czas wykonania: 10 minut 12.370882987976074 sekund
Processed 70220 record
Czas wykonania: 11 minut 13.193327188491821 sekund
Processed 70230 record
Czas wykonania: 12 minut 14.

In [26]:
for i in range(70090,70099):
    print(str(i)+' CG:'+df.loc[i, 'Wygenerowane'])


70090 CG:I was really hoping for a better experience at this restaurant, but it left much to be desired. The ambiance was bland and sort of unwelcoming. We were seated near the kitchen despite making a reservation days in advance, which made for a rather noisy meal. The menu seemed interesting at first, but the execution of the dishes we ordered was quite disappointing. My main course was not only served lukewarm but also lacked the promised flavors. The services was another area that could use improvement. While the staff was not overly rude, their attention to our needs was minimal and slow. Overall, there needs to be a significant improvement in both food quality and service for me to consider returning. This spot just didn’t live up to the hype. 2 stars.
70091 CG:I recently visited The Seaside Grill and had a delightful dining experience that deserves a solid 4-star rating. The ambiance of the restaurant was inviting with its nautical theme and soothing colors, perfectly complement

In [28]:
reviews_generated5_full = df.reset_index()
reviews_generated5_full['Wygenerowane'] = reviews_generated5_full['Wygenerowane'].replace('', np.nan)
reviews_generated5_full['text_'] = np.where(reviews_generated5_full['Wygenerowane'].notnull(), reviews_generated5_full['Wygenerowane'], reviews_generated5_full['reviewText'])
reviews_generated5_full['label'] = np.where(reviews_generated5_full['Wygenerowane'].notnull(), 'CG', 'OG')
reviews_generated5_full['target'] = np.where(reviews_generated5_full['Wygenerowane'].notnull(), 1, 0)

#[['index','reviewText', 'Wygenerowane']]
print(reviews_generated5_full)
reviews_generated4_full.to_excel('reviews_generated5_full.xlsx', index=True)
reviews_generated4_full.to_csv('reviews_generated5_full.csv', index=True)

       index  stars                                         reviewText  \
0          0      3  If you decide to eat here, just be aware it is...   
1          1      5  I've taken a lot of spin classes over the year...   
2          2      3  Family diner. Had the buffet. Eclectic assortm...   
3          3      5  Wow!  Yummy, different,  delicious.   Our favo...   
4          4      4  Cute interior and owner (?) gave us tour of up...   
...      ...    ...                                                ...   
99995  99995      5  Came in for my 5-6 month prophy and saw Kara -...   
99996  99996      3  Why 3 stars?I love Target, but here is just so...   
99997  99997      4  The Local Taco is the first joint I've found i...   
99998  99998      5  Top notch! He will cater our wedding and  Fran...   
99999  99999      5  Best place to get a vietnamese sandwich and bo...   

                                   name  \
0          Turning Point of North Wales   
1            Body Cycle S

In [29]:
reviews_generated5_cut = reviews_generated5_full[['text_', 'label', 'target']].iloc[70000:71000]
reviews_generated5_cut.columns = ['text', 'label', 'target']
reviews_generated5_cut.to_csv('reviews_generated5_cut.csv', index=True)
reviews_generated5_cut.to_excel('reviews_generated5_cut.xlsx', index=True)
print(reviews_generated5_cut)

                                                    text label  target
70000  I recently visited Cafe Belle, and I must say ...    CG       1
70001  I recently visited this charming restaurant an...    CG       1
70002  Last night, we had the pleasure of dining at T...    CG       1
70003  I had the pleasure of dining at La Bella Risto...    CG       1
70004  Dined here last night and was extremely disapp...    CG       1
...                                                  ...   ...     ...
70995  Zero if I could. So far, I've had to locate a ...    OG       0
70996  I love this movie theater and have been coming...    OG       0
70997  Swordfish was amazing. Zeppolis with Nutella w...    OG       0
70998  Cool atmosphere, nice waitstaff and the burger...    OG       0
70999  I've been to the downtown location many times ...    OG       0

[1000 rows x 3 columns]
